In [2]:
# Models
from END import EnsembleND
import NestedDichotomies.nd as nd
from sklearn.ensemble import RandomForestClassifier
from PairwiseCoupling import PairwiseCoupling
from sklearn.neural_network import MLPClassifier
# Baselearner
from sklearn import tree
from sklearn import neighbors
from sklearn.linear_model import LogisticRegression
# Methods
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import LabelEncoder
# Basics
import pandas as pd
import os
from threading import Thread
import openml
import numpy as np
from sklearn.metrics import make_scorer
import warnings;
warnings.filterwarnings('ignore');

In [3]:
def brier_score(y_predict, y_test, nclass):
    obj_num = np.size(y_test)
    bs_ytrue = np.zeros((obj_num,nclass))
    for i in range(obj_num):
        bs_ytrue[i,y_test[i]]=1
    bs = sum(sum((y_predict-bs_ytrue)**2))/obj_num
    return bs

In [4]:
from modelcombos.END_DT import EnsembleND_DT as END_DT
from modelcombos.END_LR import EnsembleND_LR as END_LR
from modelcombos.END_NB import EnsembleND_NB as END_NB
from modelcombos.PC_DT import PairwiseCoupling_DT as PC_DT
from modelcombos.PC_LR import PairwiseCoupling_LR as PC_LR
from modelcombos.PC_NB import PairwiseCoupling_NB as PC_NB
def generateModelName(model):
    name = 'error'
    if (model.__class__==RandomForestClassifier):
        name = 'RF'
    if (model.__class__==PC_DT):
        name = 'PC_DT'
    if (model.__class__==PC_LR):
        name = 'PC_LR'
    if (model.__class__==PC_NB):
        name = 'PC_NB'
    if (model.__class__==MLPClassifier):
        name = 'MLP'
    if (model.__class__==END_DT):
        name = 'END_DT'
    if (model.__class__==END_LR):
        name = 'END_LR'
    if (model.__class__==END_NB):
        name = 'END_NB'
    return name

In [5]:
def brier_score_singular_factory(nclass):
    #def brier_score_singular(y_test, y_predict):
    #    bs_ytrue = np.zeros((nclass))
    #    bs_ytrue[y_test]=1
    #    bs = sum((y_predict-bs_ytrue)**2)
    #    return bs
    def brier_score_singular(y_test,y_predict):
        #print(y_predict)
        #print(y_test)
        obj_num = np.size(y_test)
        bs_ytrue = np.zeros((obj_num,nclass))
        for i in range(obj_num):
            bs_ytrue[i,y_test[i]]=1
        bs = sum(sum((y_predict-bs_ytrue)**2))/obj_num
        return bs
    
    return brier_score_singular

In [6]:
def TMTB_and_ECE(y_predict, y_test, nclass, nbins=10, ccstrat='uniform'):
    y_pred_list = np.reshape(y_predict,nclass*y_test.size)
    ninst = y_test.size
    onehot = np.zeros((ninst, nclass))
    onehot[np.arange(ninst), y_test] = 1
    y_test_list = np.reshape(onehot,nclass*y_test.size)
    prob_true, prob_pred = calibration_curve(y_test_list, y_pred_list, n_bins=nbins, strategy=ccstrat)
    ece = np.sum(np.absolute(prob_true-prob_pred))/prob_true.size
    return ece

In [7]:
def saveCSVofData(path, file_name, data, columns):
    df = pd.DataFrame(np.array(data),columns=columns)
    df.to_csv(path+file_name+'.csv', sep='\t', encoding='utf-8', index=False)
    
def saveResults(dataset_id, bs_byModel_base_run, ece_byModel_base_run, bs_byModel_sig_run, ece_byModel_sig_run, bs_byModel_iso_run, ece_byModel_iso_run, hyperparam_byModel_run, mft_byModel_run, mst_byModel_run):
    dir_path = os.getcwd()
    directory = dir_path+'/experiments/'+str(dataset_id)+'/result_'+strftime("%Y-%m-%d %H_%M_%S", gmtime())+'/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    models = generate_ModelHyperparam_pairs()
    m_names = [generateModelName(models[i][0]) for i in range(len(models))]
    saveCSVofData(directory,'brier_base', bs_byModel_base_run, m_names)
    saveCSVofData(directory,'ece_base', ece_byModel_base_run, m_names)
    saveCSVofData(directory,'brier_sigmoid', bs_byModel_sig_run, m_names)
    saveCSVofData(directory,'ece_sigmoid', ece_byModel_sig_run, m_names)
    saveCSVofData(directory,'brier_isotonic', bs_byModel_iso_run, m_names)
    saveCSVofData(directory,'ece_isotonic', ece_byModel_iso_run, m_names)
    saveCSVofData(directory,'hyperparameter', hyperparam_byModel_run, m_names)
    saveCSVofData(directory,'mean_fit_time', mft_byModel_run, m_names)
    saveCSVofData(directory,'mean_score_time', mst_byModel_run, m_names)

In [8]:
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import RandomizedSearchCV
# Compare all models
def CompareModels(data_X, data_y, n_runs=1, n_jobs=1):
    seed = 42
    num_classes = np.unique(data_y).size
    # for each run the brierscore for each model [no calibration]
    bs_byModel_base_run = []
    # for each run the ece (expected calibration error) for each model [no calibration]
    ece_byModel_base_run = []
    # for each run the brierscore for each model [sigmoid calibration]
    bs_byModel_sig_run = []
    # for each run the ece (expected calibration error) for each model [sigmoid calibration]
    ece_byModel_sig_run = []
    # for each run the brierscore for each model [isotonic calibration]
    bs_byModel_iso_run = []
    # for each run the ece (expected calibration error) for each model [isotonic calibration]
    ece_byModel_iso_run = []
    # for each run the hyperparameter for each model
    hyperparam_byModel_run = []
    # for each run the mean fit time for each model
    mft_byModel_run = []
    # for each run the mean score time for each model
    mst_byModel_run = []
    # RUNS
    for i in range(n_runs):
        print('run',i+1)
        bs_byModel_base_run.append([])
        ece_byModel_base_run.append([])
        bs_byModel_sig_run.append([])
        ece_byModel_sig_run.append([])
        bs_byModel_iso_run.append([])
        ece_byModel_iso_run.append([])
        hyperparam_byModel_run.append([])
        mft_byModel_run.append([])
        mst_byModel_run.append([])
        # train - test von data (80/20)
        X_train, X_test, y_train, y_test = tts(data_X, data_y, test_size=0.2, stratify=data_y, random_state=seed)
        # model - calibration split von train (70/30)
        X_model, X_calibration, y_model, y_calibration = tts(X_train, y_train, test_size=0.3, stratify=y_train, random_state=seed+1)
        
        scoring = {'bs': make_scorer(brier_score_singular_factory(num_classes), greater_is_better=False, needs_proba=True)}
        
        model_dists = generate_ModelHyperparam_pairs(num_classes, seed)
        
        for k in range(len(model_dists)):
            print('model',k+1)
            #if (k==1):
            #    set_trace()
            model_rs = RandomizedSearchCV(model_dists[k][0], param_distributions=model_dists[k][1], scoring=scoring,refit='bs', n_iter = 10, n_jobs = n_jobs, cv=3, random_state=seed)
            model_rs.fit(X_model, y_model)
            print('mean fit time:',model_rs.cv_results_['mean_fit_time'].mean())
            print('mean score time:',model_rs.cv_results_['mean_score_time'].mean())
            seed += 1
            print(model_rs.best_params_)
            # best estimator
            hyperparam_byModel_run[i].append(model_rs.best_params_)
            model = model_rs.best_estimator_
            #calibration sigmoid
            c_sig_model = CalibratedClassifierCV(base_estimator=model,method='sigmoid', cv='prefit')
            c_sig_model.fit(X_calibration, y_calibration)
            #calibration isotonic
            c_iso_model = CalibratedClassifierCV(base_estimator=model,method='isotonic', cv='prefit')
            c_iso_model.fit(X_calibration, y_calibration)
            #prediction base
            y_pred_base = model.predict_proba(X_test)
            bs_base = brier_score(y_predict=y_pred_base, y_test=y_test, nclass=num_classes)
            ece_base = TMTB_and_ECE(y_predict=y_pred_base, y_test=y_test, nclass=num_classes, nbins=10, ccstrat='uniform')
            #prediction sigmoid calibrated model
            y_pred_sig = c_sig_model.predict_proba(X_test)
            bs_sig = brier_score(y_predict=y_pred_sig, y_test=y_test, nclass=num_classes)
            ece_sig = TMTB_and_ECE(y_predict=y_pred_sig, y_test=y_test, nclass=num_classes, nbins=10, ccstrat='uniform')
            #prediction isotonic calibrated model
            y_pred_iso = c_iso_model.predict_proba(X_test)
            bs_iso = brier_score(y_predict=y_pred_iso, y_test=y_test, nclass=num_classes)
            ece_iso = TMTB_and_ECE(y_predict=y_pred_iso, y_test=y_test, nclass=num_classes, nbins=10, ccstrat='uniform')
            #append results
            bs_byModel_base_run[i].append(bs_base)
            ece_byModel_base_run[i].append(ece_base)
            bs_byModel_sig_run[i].append(bs_sig)
            ece_byModel_sig_run[i].append(ece_sig)
            bs_byModel_iso_run[i].append(bs_iso)
            ece_byModel_iso_run[i].append(ece_iso)
            mft_byModel_run[i].append(model_rs.cv_results_['mean_fit_time'].mean())
            mst_byModel_run[i].append(model_rs.cv_results_['mean_score_time'].mean())
            print(bs_base,ece_base,bs_sig, ece_sig, bs_iso, ece_iso)
    return bs_byModel_base_run, ece_byModel_base_run, bs_byModel_sig_run, ece_byModel_sig_run, bs_byModel_iso_run, ece_byModel_iso_run, hyperparam_byModel_run, mft_byModel_run, mst_byModel_run

In [9]:
from modelcombos.END_DT import EnsembleND_DT as END_DT
from modelcombos.END_LR import EnsembleND_LR as END_LR
from modelcombos.END_NB import EnsembleND_NB as END_NB
from modelcombos.PC_DT import PairwiseCoupling_DT as PC_DT
from modelcombos.PC_LR import PairwiseCoupling_LR as PC_LR
from modelcombos.PC_NB import PairwiseCoupling_NB as PC_NB
from scipy.stats import randint
from scipy.stats import uniform
def generate_ModelHyperparam_pairs(nclasses=3, seed=42):
    models = []
    RFC_paramdist =  {
        'min_impurity_decrease': uniform(0.00001, 0.1),
        'min_samples_leaf': randint(1,51)}
    models.append((RandomForestClassifier(n_estimators=45, random_state=seed), RFC_paramdist))
    DT_paramdist = {
        'max_depth': randint(1,4)
    }
    LR_paramdist = {
        'penalty': ['l1', 'l2'],
        'C': uniform(0.01,20)
    }
    NB_paramdist = {
        'var_smoothing' : uniform(0.0000000001,1.0)
    }
    models.append((END_DT(number_of_nds=5, number_of_classes=nclasses, max_depth = 1, generator_String='random_pair', random_state=seed), DT_paramdist))
    models.append((END_LR(number_of_nds=5, number_of_classes=nclasses,penalty='l2', C=1.0, generator_String='random_pair', random_state=seed), LR_paramdist))
    models.append((END_NB(number_of_nds=5, number_of_classes=nclasses, var_smoothing=0.00001, generator_String='random_pair', random_state=seed),NB_paramdist))
    models.append((PC_DT(classes=nclasses, seed=seed, max_depth=1),DT_paramdist))
    models.append((PC_LR(classes=nclasses, seed=seed, penalty='l2', C=1.0),LR_paramdist))
    models.append((PC_NB(classes=nclasses, seed=seed, var_smoothing=0.0001),NB_paramdist))
    MLP_paramdist = {
        'alpha': uniform(0.00000001, 5),
        'batch_size': [100,200,300,400,500],
        'power_t': uniform(0.001,3) 
    }
    models.append((MLPClassifier(hidden_layer_sizes = (50,50,), activation='relu', learning_rate='invscaling', max_iter=1000), MLP_paramdist))
    return models

In [10]:
from time import gmtime, strftime
    
def write_single_value_per_run(file, models, value_name, values):
    file.write(value_name+'\n')
    file.write(generateModelName(models[0][0]))
    for i in range(len(models)-1):
        file.write('\t'+generateModelName(models[i+1][0]))
    file.write('\n')
    for i in range(len(values)):
        file.write(str(values[i][0]))
        for k in range(len(values[i])-1):
            file.write('\t'+str(values[i][k+1]))
        file.write('\n')
    
def save_result(dataset_id, nclasses, bs_byModel_base_run, ece_byModel_base_run, bs_byModel_sig_run, ece_byModel_sig_run, bs_byModel_iso_run, ece_byModel_iso_run, hyperparam_byModel_run, mft_byModel_run, mst_byModel_run):
    dir_path = os.getcwd()
    directory = dir_path+'/experiments/'+dataset_id+'/'
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    models = generate_ModelHyperparam_pairs(nclasses=nclasses, seed=42)
    file = open(directory+'result_'+strftime("%Y-%m-%d %H_%M_%S", gmtime())+'.txt', 'w')
    # WRITE MODEL DESCRIPTIONS
    try:
        # SAVING BRIER SCORE
        file.write('Brier-Score\n')
        file.write(generateModelName(models[0][0]))
        for i in range(len(models)-1):
            file.write('\t\t\t'+generateModelName(models[i+1][0]))
        file.write('\nBase\tSigmoid\tIsotonic')
        for i in range(len(models)-1):
            file.write('\tBase\tSigmoid\tIsotonic')
        file.write('\n')
        for i in range(len(bs_byModel_base_run)):
            file.write(str(bs_byModel_base_run[i][0])+'\t'+str(bs_byModel_sig_run[i][0])+'\t'+str(bs_byModel_iso_run[i][0]))
            for k in range(len(bs_byModel_base_run[i])-1):
                file.write('\t'+str(bs_byModel_base_run[i][k+1])+'\t'+str(bs_byModel_sig_run[i][k+1])+'\t'+str(bs_byModel_iso_run[i][k+1]))
            file.write('\n')
        # SAVING ECE SCORE
        file.write('ECE-Score\n')
        file.write(generateModelName(models[0][0]))
        for i in range(len(models)-1):
            file.write('\t\t\t'+generateModelName(models[i+1][0]))
        file.write('\nBase\tSigmoid\tIsotonic')
        for i in range(len(models)-1):
            file.write('\tBase\tSigmoid\tIsotonic')
        file.write('\n')
        for i in range(len(ece_byModel_base_run)):
            file.write(str(ece_byModel_base_run[i][0])+'\t'+str(ece_byModel_sig_run[i][0])+'\t'+str(ece_byModel_iso_run[i][0]))
            for k in range(len(ece_byModel_base_run[i])-1):
                file.write('\t'+str(ece_byModel_base_run[i][k+1])+'\t'+str(ece_byModel_sig_run[i][k+1])+'\t'+str(ece_byModel_iso_run[i][k+1]))
            file.write('\n')
        # SAVING HYPERPARAMETER
        write_single_value_per_run(file, models, 'Hyperparameter', hyperparam_byModel_run)
        # SAVING MEAN FIT TIME
        write_single_value_per_run(file, models, 'Mean Fit Time', mft_byModel_run)
        # SAVING MEAN SCORE TIME
        write_single_value_per_run(file, models, 'Mean Score Time', mst_byModel_run)
    finally:
        file.close()

In [11]:
#1515 - micro-mass
#1459 - artificial-characters
#1233 - eating
#1569 - poker-hand
#1503 - spoken-arabic-digit
#4541 - Diabetes130US
#4538 - GesturePhaseSegmentationProcessed
#41991 - Kuzushiji-49 - error
#40670 - dna
#1478 - har
#40984 - segment
#40498 - wine-quality-white
#40499 - texture
#40686 - solar-flare
#41972 - Indian_pines
#1475 - first-order-theorem-proving
#id_list = [1515,1459,1233,1569,1503,4541,4538,41991,40670,1478,40984,40498,40499,40686,41972,1475]
#for i in range(len(id_list)):
#    ID_n = id_list[i]
#    RUNS_n = 20
#    JOBS_n = 10
#    run_dataset(ID_n, RUNS_n, JOBS_n)

In [12]:
# start experiment
def run_dataset(ID, RUNS, JOBS):
    dataset = openml.datasets.get_dataset(ID)
    dataset_name = dataset.name
    dataset_id = dataset.dataset_id
    print(dataset_name, 'id: ', dataset_id)
    X, y, categorical_indicator, attribute_names = dataset.get_data(dataset_format='array', target=dataset.default_target_attribute)
    print(np.unique(y))
    num_classes = np.unique(y).size
    #bs & ece for (b)ase, (s)igmoid and (i)sotonic
    bs_b, ece_b, bs_s, ece_s, bs_i, ece_i, hyppar, mft, mst = CompareModels(data_X=X, data_y=y, n_runs=RUNS, n_jobs=JOBS)
    saveResults(dataset_id=dataset_id, bs_byModel_base_run=bs_b, ece_byModel_base_run=ece_b, bs_byModel_sig_run=bs_s, ece_byModel_sig_run=ece_s, bs_byModel_iso_run=bs_i, ece_byModel_iso_run=ece_i, hyperparam_byModel_run=hyppar, mft_byModel_run=mft, mst_byModel_run=mst)
    #save_result(dataset_name=dataset_name, nclasses=num_classes, bs_byModel_base_run=bs_b, ece_byModel_base_run=ece_b, bs_byModel_sig_run=bs_s, ece_byModel_sig_run=ece_s, bs_byModel_iso_run=bs_i, ece_byModel_iso_run=ece_i, hyperparam_byModel_run=hyppar, mft_byModel_run=mft, mst_byModel_run=mst)

In [ ]:
id_list = [1515,1459,1233,1569,1503,4541,4538,40670,1478,40984,40498,40499,40686,41972,1475,40474,1565,1559,1568,1560,40668,11,62,2,1557,4153,40975,61,300,1509]
print(len(id_list))
print(len(np.unique(id_list)))
for i in range(24,len(id_list)):
    ID_n = id_list[i]
    RUNS_n = 20
    JOBS_n = 10
    run_dataset(ID_n, RUNS_n, JOBS_n)

30
30
abalone id:  1557
[0 1 2]
run 1
model 1
mean fit time: 0.08721036911010742
mean score time: 0.008754483858744304
{'min_impurity_decrease': 8.787658410143284e-05, 'min_samples_leaf': 21}
0.4702741000350361 0.035355082130345425 0.4756820537552306 0.04757741564650846 0.472405129979134 0.022905308145334098
model 2
mean fit time: 0.04412058194478353
mean score time: 0.0034865538279215498
{'max_depth': 3}
0.511555680484794 0.04292708369624995 0.5148050750673303 0.04155747852106613 0.5108825715267518 0.0557276558510227
model 3
mean fit time: 0.773821767171224
mean score time: 0.0036421616872151694
{'C': 14.21295986654147, 'penalty': 'l1'}
0.48052999614427006 0.028019917865677223 0.4868581217469265 0.041475875418297226 0.48392105584844897 0.02086800855408885
model 4
mean fit time: 0.05591962337493896
mean score time: 0.016459377606709798
{'var_smoothing': 0.16332444830826676}
0.5310067709969344 0.044416023689073755 0.5306527075226061 0.05280223226963788 0.5289280602618014 0.0312398960457

mean fit time: 1.591879455248515
mean score time: 0.003910001118977864
{'C': 13.136117178655478, 'penalty': 'l1'}
0.4493129472095677 0.021405101755760415 0.4508156484978523 0.02657971631488558 0.4583699687319592 0.03892836349431908
model 4
mean fit time: 0.0330948273340861
mean score time: 0.0072480599085489915
{'var_smoothing': 0.24023517596800603}
0.5225582178126821 0.039741819128894865 0.5224048336096015 0.04979265692402896 0.5207447559086149 0.05374036069910304
model 5
mean fit time: 0.0070400079091389985
mean score time: 0.2631131331125895
{'max_depth': 3}
0.4737336774137129 0.030488727697462876 0.4758801654903811 0.025897464373502217 0.46854375125851905 0.025298330183604036
model 6
mean fit time: 0.14680760701497395
mean score time: 0.26682790915171306
{'C': 9.241285229441898, 'penalty': 'l1'}
0.4495493124893721 0.028416435569736255 0.45041764305668114 0.029394884694522908 0.45710247767202367 0.04987888109419709
model 7
mean fit time: 0.005939316749572755
mean score time: 0.61828

mean fit time: 0.007816894849141439
mean score time: 0.28944608370463054
{'max_depth': 3}
0.47870361282124396 0.06826864023494929 0.481488649392107 0.06385239384043481 0.47079253235717594 0.03466590503313858
model 6
mean fit time: 0.15385483105977377
mean score time: 0.2738449017206828
{'C': 15.395249450462456, 'penalty': 'l1'}
0.45017681421595945 0.0281036523314089 0.4536020943429155 0.030163671870177558 0.45043238598767543 0.03133714904526951
model 7
mean fit time: 0.006356565157572427
mean score time: 0.623556383450826
{'var_smoothing': 0.37505712897328514}
0.5220784401861683 0.07067551863362426 0.5174981511285748 0.04157489681800167 0.5113732045134419 0.047847986800414063
model 8
mean fit time: 2.8692528883616126
mean score time: 0.0031083265940348307
{'alpha': 0.5423244362959421, 'batch_size': 400, 'power_t': 0.23412423749074507}
0.43335837619378803 0.02003075718462379 0.4374429328729804 0.032573560142801085 0.4367262736995162 0.015620846591391022
run 10
model 1
mean fit time: 0.0

mean fit time: 0.006005827585856121
mean score time: 0.6345127344131469
{'var_smoothing': 0.29517174282345104}
0.5259546190939022 0.0331353908376384 0.5233704575368716 0.02947566058183764 0.5210659568694201 0.027318758553279344
model 8
mean fit time: 2.526530210177104
mean score time: 0.0032848676045735673
{'alpha': 0.3857538810996816, 'batch_size': 100, 'power_t': 1.4910456749469057}
0.4385701325831539 0.030603282279137296 0.4422057494330423 0.031001149892698563 0.44457420837365536 0.023827393927100166
run 14
model 1
mean fit time: 0.07957646052042643
mean score time: 0.008088143666585286
{'min_impurity_decrease': 0.026080283762706193, 'min_samples_leaf': 2}
0.5446842072130403 0.06526952147549633 0.5455443546638296 0.0926368035843089 0.5055793490597791 0.019681227999498757
model 2
mean fit time: 0.06261937618255616
mean score time: 0.005942130088806152
{'max_depth': 3}
0.4811355109120897 0.032706532749588235 0.48548820781028007 0.06100298536363179 0.4787298055964076 0.0258544644542015

mean fit time: 0.07895297209421792
mean score time: 0.008005324999491373
{'min_impurity_decrease': 0.007511710495915126, 'min_samples_leaf': 13}
0.497364992285026 0.0435697324304025 0.49421989675934536 0.04035229359075428 0.48832241833447554 0.026516218014099945
model 2
mean fit time: 0.04466548760732015
mean score time: 0.0034697771072387694
{'max_depth': 3}
0.48496759664256767 0.04352697842880128 0.48410531954401614 0.03176525771504952 0.4746908167732435 0.02460382316424623
model 3
mean fit time: 0.9686783154805502
mean score time: 0.004213404655456542
{'C': 5.408818256936466, 'penalty': 'l1'}
0.4526169999500146 0.024292644850459632 0.45580734743461393 0.036404477039690866 0.453724107194308 0.024437869127659762
model 4
mean fit time: 0.03360342184702555
mean score time: 0.007435727119445801
{'var_smoothing': 0.1821341143743488}
0.5157804054010375 0.03456311603505903 0.5154314955638873 0.04287176810120327 0.5101995449134719 0.026360671940764257
model 5
mean fit time: 0.009020892779032

...

0.06572263480733354 0.035168779612230285 0.07502212112244187 0.19275153044270163 0.06720522134765365 0.04074745877968157
run 6
model 1
mean fit time: 0.4549193779627482
mean score time: 0.0407593011856079
{'min_impurity_decrease': 0.013415000147746449, 'min_samples_leaf': 26}
0.5551393116147829 0.30239822347023915 0.28760545947216687 0.10781767919898219 0.2840394930434896 0.06779699718074886
model 2
mean fit time: 13.821928596496582
mean score time: 0.13144487539927166
{'max_depth': 3}
0.2576395541803342 0.12941445817514152 0.20670769039353493 0.03812153012608007 0.20505696016473704 0.04065105526037795
model 3
mean fit time: 22.72450122833252
mean score time: 0.4134367386500041
{'C': 3.481783266932984, 'penalty': 'l2'}
0.06986708929749004 0.06179393614584126 0.06956097007075887 0.13690416728207372 0.0750889032187387 0.04718493309730867
model 4
mean fit time: 10.821766789754232
mean score time: 4.978095602989198
{'var_smoothing': 0.2735065344522666}
0.402479332598302 0.08068618450631962

0.25774135629116457 0.12499951930434605 0.21243768625850637 0.05173621467363525 0.2124864335418006 0.02715731058844744
model 3
mean fit time: 23.709039711952208
mean score time: 0.5852769692738852
{'C': 15.400529916494804, 'penalty': 'l2'}
0.07465363243734303 0.05873140986582749 0.07612495690136757 0.10297150204746774 0.07717861694050304 0.0930192942238735
model 4
mean fit time: 12.34105672836304
mean score time: 5.050218764940898
{'var_smoothing': 0.04540221017700707}
0.32571575220836296 0.07595694048725923 0.28058006284888454 0.03197220505220566 0.2596933732103203 0.030754890516677864
model 5
mean fit time: 4.567808039983113
mean score time: 45.97618815104166
{'max_depth': 3}
0.2806461989961265 0.1389167143766138 0.27892173016585553 0.16435771294038343 0.2343637725979791 0.06918133734733949
model 6
mean fit time: 4.083355196317037
mean score time: 49.071604530016586
{'C': 19.30564111526844, 'penalty': 'l2'}
0.0652597071527371 0.05686724906978633 0.07509008607250843 0.1845764715614906

mean fit time: 4.594383160273234
mean score time: 45.1819935242335
{'max_depth': 3}
0.28830651363049564 0.1740889516631046 0.27847065711414615 0.10824636405953242 0.2460046305843282 0.0886113867722013
model 6
mean fit time: 3.8529765764872237
mean score time: 47.598100344340004
{'C': 16.01627754148761, 'penalty': 'l2'}
0.06585703028157103 0.044567838883161924 0.0736187313272124 0.222655277066853 0.06956905904750205 0.05333484649149356
model 7
mean fit time: 0.8742335557937622
mean score time: 135.44033877054852
{'var_smoothing': 0.03832907311201927}
0.21873070850290655 0.2465736375596383 0.20901706267684206 0.30209850658888926 0.17517786048672607 0.04412540484633098
model 8
mean fit time: 16.677731823921203
mean score time: 0.020548081398010253
{'alpha': 2.501912554754114, 'batch_size': 500, 'power_t': 0.22438881708295533}
0.08213980721444501 0.07731300425081537 0.07685433691648628 0.06278054872088416 0.07542609903135032 0.014632791723353394
run 15
model 1
mean fit time: 0.692895936965

0.2125487588220655 0.21931733558126093 0.2044987377618544 0.2829365493007029 0.17296486915755976 0.03582698776209372
model 8
mean fit time: 21.457380827267965
mean score time: 0.026355703671773277
{'alpha': 0.12121788431644603, 'batch_size': 200, 'power_t': 2.2213922645774136}
0.07132422196889858 0.06216488311027483 0.0833061793836915 0.18549954142291136 0.07077268556404961 0.05770488661739143
run 19
model 1
mean fit time: 0.7338489135106404
mean score time: 0.038644266128540036
{'min_impurity_decrease': 0.006940047351797297, 'min_samples_leaf': 40}
0.4169412676560657 0.2756470926889156 0.19286581300296185 0.062219518317876574 0.20195302821544345 0.04905905946272955
model 2
mean fit time: 18.009340890248616
mean score time: 0.1354140758514404
{'max_depth': 3}
0.22105884765646905 0.12379178511646008 0.1829625542866598 0.06142595060968038 0.18427988325736747 0.04625849727577366
model 3
mean fit time: 32.3311225493749
mean score time: 0.6666870911916096
{'C': 10.249552349461576, 'penalty'

0.8019482024000176 0.142738367790115 0.7727448129434902 0.03404736327508321 0.751739663755415 0.03929914880893705
model 2
mean fit time: 94.99694197177887
mean score time: 0.24898783365885416
{'max_depth': 3}
0.7238702860753398 0.07874400873231334 0.7251367262256665 0.061356791147773544 0.692753899994876 0.041098997070483166
model 3
mean fit time: 96.40038339296976
mean score time: 0.24474229017893473
{'C': 2.4778674136020804, 'penalty': 'l2'}
0.8259064543948745 0.18854472849034312 0.8354868562342946 0.1548532237454266 0.7993663067686703 0.13936600404786387
model 4
mean fit time: 120.23574508825936
mean score time: 0.6451785643895468
{'var_smoothing': 0.0035455469097308485}
0.8488246199739471 0.03483789176029 0.8436917251945418 0.05997968389477145 0.824604988136245 0.08912614398415761
model 5
mean fit time: 1.7236159960428874
mean score time: 573.5876481215158
{'max_depth': 3}
0.6884514589146951 0.04267851794230205 0.6968868797957903 0.04836408611223234 0.6693766279669701 0.02809277246

In [ ]:
#id_list = [1515,1459,1233,1569,1503,4541,4538,40670,1478,40984,40498,40499,40686,41972,1475,40474,1565,1559,1568,1560,40668,11,62,2,1557,4153,40975,61,300,1509]
#print(len(id_list))
#print(len(np.unique(id_list)))
#for i in range(0,len(id_list)):
#    dataset = openml.datasets.get_dataset(id_list[i])
#    dataset_name = dataset.name
#    dataset_id = dataset.dataset_id
#    print(dataset_name, dataset_id)
#    X, y, categorical_indicator, attribute_names = dataset.get_data(dataset_format='array', target=dataset.default_target_attribute)
#    print(np.unique(y))